### Hierarchical Bias-Aware Clustering (HBAC)

In this notebook we experimented with HBAC according to our interpretation of the paper [Auditing a dutch public sector risk profiling algorithm using an unsupervised bias detection tool](https://arxiv.org/pdf/2502.01713). Due to the time constrain of this project and the lack of clarity in the paper on certain details of an implementation, we are not certain if our implementation is what the author of the paper had in mind. We suggest that those who are interrested in this subject do further research and experiment for themselves.

In our implementation K-modes is used for making the clusters, which causes the interpretability of the algorithm's output to be difficult. We made an atempt at an example interpretation but we cannot be certain this is correct due to a lack of transparency in the prediction of K-modes.

In [9]:
import numpy as np
import pandas as pd
from collections import deque
from kmodes.kmodes import KModes
import plotly.graph_objects as go

import torch

from fairness_metrics.Predicted_outcomes.statistical_parity import statistical_parity

loaded statisctical


In [ ]:
df = pd.read_csv("data/altered_data/data_pred_ground_altered_pred.csv")

df['bin_predictions'] = [1 if i > 0.7 else 0 for i in df['predictions']] # For probability predictions -> above 0.7 will be classified as a 1 else 0

# Features taken into account when clustering with K-modes
features = [col for col in df.columns if col !='predictions' and col !='bin_predictions' and col !='actual_outcome' and col !='cluster' and col != 'Unnamed: 0']
X = df[features].astype(str).values
y = df['bin_predictions'].values

# Iterations of clustering algorithm
max_iter = 10

# Smallest number of samples a cluster is allowed to have
min_size = 50

# The collumn we look at when computing the bias
bias_test_cols = 'persoon_geslacht_vrouw'


class Dataset:
    """
    Class for structuring the dataset
    """
    def __init__(self, df):
        data = torch.tensor(df.values, dtype=torch.float)
        self.data = data
        self.columns = df.columns.tolist()
        self.i2c = self.columns
        self.c2i = {name: i for i, name in enumerate(self.columns)}


class ClusterNode:
    """
    Class to keep track of the data for each cluster
    """
    def __init__(self, indices, cluster_id):
        self.indices = indices
        self.left = None
        self.right = None
        self.cluster_id = cluster_id


def compute_bias(sub_df, bias_test_cols):
    """
    Computes biggest difference in positive prediction chance between all group of the given collumn
    """
    wrapped = Dataset(sub_df)

    attr = bias_test_cols
    params = {
        'prediction_column': 'bin_predictions',
        'ground_truth_column': 'ground_truth',
        'protected_values': torch.tensor([col == attr for col in sub_df.columns])
    }

    metric = statistical_parity(wrapped, params)
    results = metric.show(raw_results=True)

    group_probs = results[attr]['group_probs']
    probs = list(group_probs.values())
    assert len(probs) >= 2, 'The chosen attribute to test fairness on has only one group, thus we cannot calculate the difference'
    diff = abs(max(probs) - min(probs))
    return diff

        

def hbca_tree(X, y, df, max_iter, min_size=50):
    """
    Clustering algorithm which uses K-modes to make clusters which are then compared and 
    """
    cluster_counter = 1
    root = ClusterNode(np.arange(len(y)), cluster_id=0)
    queue = deque([root])

    df['cluster'] = -1
    df.loc[df.index[root.indices], 'cluster'] = 0

    cluster_spd_log = {}

    for _ in range(max_iter):
        if queue:
            node = queue.popleft()

            if len(node.indices) >= 2 * min_size:
                kmodes = KModes(n_clusters=2)
                labels = kmodes.fit_predict(X[node.indices]) # K-modes to split the data on the features

                left_indices = node.indices[labels == 0]
                right_indices = node.indices[labels == 1]

                parent_df = df.iloc[node.indices]
                left_df = df.iloc[left_indices]
                right_df = df.iloc[right_indices]

                # calculate bias in the sets
                spd_parent = compute_bias(parent_df, bias_test_cols)
                spd_left = compute_bias(left_df, bias_test_cols)
                spd_right = compute_bias(right_df, bias_test_cols)

                # if there is more bias in a split, add the splits to the data and remove the set before splitting
                if max(spd_left, spd_right) > spd_parent and (len(left_indices) >= min_size or len(right_indices) >= min_size):
                    left_id = cluster_counter
                    right_id = cluster_counter + 1
                    cluster_counter += 2

                    df.loc[df.index[left_indices], 'cluster'] = left_id
                    df.loc[df.index[right_indices], 'cluster'] = right_id

                    node.left = ClusterNode(left_indices, cluster_id=cluster_counter - 2)
                    node.right = ClusterNode(right_indices, cluster_id=cluster_counter - 1)

                    queue.extend([node.left, node.right])

                    if node.cluster_id in cluster_spd_log:
                        cluster_spd_log.__delitem__(node.cluster_id)

                    cluster_spd_log[left_id] = spd_left
                    cluster_spd_log[right_id] = spd_right
                else:
                    # If there isn't more bias in a split, keep the non-splitted set
                    cluster_spd_log[node.cluster_id] = spd_parent

    return df, cluster_spd_log


df, cluster_spd_log = hbca_tree(X, y, df, max_iter, min_size) # run algorithm


# 
sorted_clusters = sorted(cluster_spd_log.items())
cluster_ids = [str(k) for k, _ in sorted_clusters]
spd_values = [v for _, v in sorted_clusters]

fig = go.Figure()

fig.add_trace(go.Bar(
    x=cluster_ids,
    y=spd_values,
    name='Statistical Parity Difference',
))

fig.update_layout(
    title='SPD per Cluster in Final HBCA Tree',
    xaxis_title='Cluster ID',
    yaxis_title='Statistical Parity Difference',
)

fig.show()

### Example interpretation

In the example given above we look at the bias between man and women and split the data when there is more bias in a split of a cluster than the cluster itself. By analysing the clusters where the differences between man and women are high we might find values of features that correspond to a increase in bias. If for example, an certain nationality is more prominent in a cluster with high bias, we can conclude that man and women of that nationality are not treated equally.